**import dataset**

In [1]:
!git clone https://github.com/elaamranisoufiane/Comp2018DatasetCleanISR.git

Cloning into 'Comp2018DatasetCleanISR'...
remote: Enumerating objects: 1330, done.
remote: Total 1330 (delta 0), reused 0 (delta 0), pack-reused 1330
Receiving objects: 100% (1330/1330), 98.33 MiB | 32.21 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

 

**performance metrics**


In [3]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

**Our Model**


In [4]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    s = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(s)
    return s, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
   
    #NEW U NET
    """ Input layer """
    inputs = Input(input_shape)
    """ Encoder """
    s1, p1 = encoder_block(inputs, 16)
    s2, p2 = encoder_block(p1, 32)
    s3, p3 = encoder_block(p2, 64)
    s4, p4 = encoder_block(p3, 128)
    s5, p5 = encoder_block(p4, 256)
    s6, p6 = encoder_block(p5, 512)
    
    """ Bottleneck """
    b1 = conv_block(p6, 1024)
    
    """ Decoder """
    d1 = decoder_block(b1, s6, 512)
    d2 = decoder_block(d1, s5, 256)
    d3 = decoder_block(d2, s4, 128)
    d4 = decoder_block(d3, s3, 64)
    d5 = decoder_block(d4, s2, 32)
    d6 = decoder_block(d5, s1, 16)
    """ Output layer """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d6)


    model = Model(inputs, outputs, name="UNET")
    return model

if __name__ == "__main__":
    model = build_unet((1024, 1024, 3))
    model.summary()
    

2023-02-13 00:30:20.578908: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-13 00:30:20.721653: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-13 00:30:20.722378: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-13 00:30:20.724025: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "UNET"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1024, 1024,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 1024, 1024, 1 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 1024, 1024, 1 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 1024, 1024, 1 0           batch_normalization[0][0]        
_______________________________________________________________________________________________

In [5]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from sklearn.model_selection import KFold
from tqdm import tqdm
import pandas as pd

#import build_unet
#import dice_coef, iou
 
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score


# input image size
H = 1024
W = 1024


# create directory and gives the path
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)


def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x


def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x


def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y



def tf_dataset(X, Y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

##########################################

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def read_image1(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x / 255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)  ## (1, 256, 256, 3)
    return ori_x, x


def read_mask1(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x / 255.0
    x = x > 0.5
    x = x.astype(np.int32)
    return ori_x, x

def save_result(ori_x, ori_y, y_pred, save_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)  ## (256, 256, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)  ## (256, 256, 3)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255.0

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_path, cat_images)

def load_data(path, split=0.2):
    images = sorted(glob(os.path.join(path, "images", "*.jpg")))
    masks = sorted(glob(os.path.join(path, "Masks", "*.jpg")))
    size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=size, random_state=42) 
    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


In [6]:
if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)
    # create a file in the same ocation
    create_dir('files')
    """ Hyperparaqmeters """
    batch_size = 8
    lr = 1e-4  ## 0.0001
    num_epochs = 260
    model_path = "files/model.h5"
    csv_path = "files/data.csv"
    create_dir("results")
    """ Dataset """
    #dataset_path = "./DBA"
    # colab
    # dataset_path = "/content/U-Net-for-Nuclei-Semantic-Segmentation/DSB"
    # kuggle
    dataset_path = "./Comp2018DatasetCleanISR"
    ##########################
    
    # Define per-fold score container
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)
    '''
    train_x = pd.DataFrame(train_x)
    test_x = pd.DataFrame(test_x)
    train_x = pd.concat([train_x, test_x]) 
    
    train_y = pd.DataFrame(train_y)
    test_y = pd.DataFrame(test_y)
    train_y = pd.concat([train_y, test_y]) 
    '''
    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")
    
    acc_per_fold = []
    loss_per_fold = []
    acc_per_fold = []
    f1_value_per_fold = []
    jac_value_per_fold = []
    recall_value_per_fold = []
    precision_value_per_fold = []
  
    
    num_folds=4
    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=num_folds, shuffle=True)
    # K-fold Cross Validation model evaluation
    fold_no = 1
    for train, test in kfold.split(train_x, train_y):
        print(f'Training for fold {fold_no} ...')
        test_x1=[]
        test_y1=[]
        for test_i in test:
            test_x1.append(train_x[test_i])
            test_y1.append(train_y[test_i])
        train_x1=[]
        train_y1=[]
        
        for train_i in train:
            train_x1.append(train_x[train_i])
            train_y1.append(train_y[train_i])
        #fold_no = fold_no + 1
        
        
        #(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)
        #(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path) 
        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')
        #train_x1, train_y1 = shuffle(train_x1, train_y1)
        print(f"Train: {len(train_x1)} - {len(train_y1)}")
        print(f"Valid: {len(valid_x)} - {len(valid_y)}")
        print(f"Test: {len(test_x1)} - {len(test_y1)}")
        train_dataset = tf_dataset(train_x1, train_y1, batch=batch_size)
        valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)
        train_steps = (len(train_x1) // batch_size)
        valid_steps = (len(valid_x) // batch_size)
        if len(train_x1) % batch_size != 0:
            train_steps += 1
        #if len(valid_x) % batch_size != 0:
            #valid_steps += 1

        for x, y in valid_dataset:
            print(x.shape, y.shape)
            break
        """ build the model with U-Net network archtecture """
        model = build_unet((H, W, 3))
        metrics = ["accuracy", dice_coef, iou, Recall(), Precision()]
        model.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=metrics)
        callbacks = [
            ModelCheckpoint(model_path, verbose=1, save_best_only=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
            CSVLogger(csv_path),
            # EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
        ]
        history = model.fit(
            train_dataset,
            epochs=num_epochs,
            validation_data=valid_dataset,
            steps_per_epoch=train_steps,
            validation_steps=valid_steps,
            callbacks=callbacks,
            shuffle=False
        )
        fold_no = fold_no + 1
        ### evaluation
        #(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)
        """ Load Model """
        with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
            model = tf.keras.models.load_model(model_path)
            #model.summary()
        """ Prediction and metrics values """
        SCORE = []
        print(f'test image : {test_x1[1]}')
        for x, y in tqdm(zip(test_x1, test_y1), total=len(test_x1)):
            name = x.split("/")[-1]
            
            """ Reading the image and mask """
            ori_x, x = read_image1(x)
            ori_y, y = read_mask1(y)
            """ Prediction """
            y_pred = model.predict(x)[0] > 0.5
            y_pred = np.squeeze(y_pred, axis=-1)
            y_pred = y_pred.astype(np.int32)
            
            save_path = f"results/{name}"
            save_result(ori_x, ori_y, y_pred, save_path)
            
            """ Flattening the numpy arrays. """
            y = y.flatten()
            y_pred = y_pred.flatten()
            
            """ Calculating metrics values """
            acc_value = accuracy_score(y, y_pred)
            f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
            jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
            recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
            precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
            SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])
           
    
        
            """ Metrics values """
        print("eval metrics")
        score = [s[1:] for s in SCORE]
        score = np.mean(score, axis=0)
        print(f"Accuracy: {score[0]:0.5f}")
        print(f"F1: {score[1]:0.5f}")
        print(f"Jaccard: {score[2]:0.5f}")
        print(f"Recall: {score[3]:0.5f}")
        print(f"Precision: {score[4]:0.5f}")
        
        """ save the results  """
        acc_per_fold.append(score[0])
        f1_value_per_fold.append(score[1])
        jac_value_per_fold.append(score[2])
        recall_value_per_fold.append(score[3])
        precision_value_per_fold.append(score[4])
           

        """ Saving all the results """
        df = pd.DataFrame(SCORE, columns=["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
        df.to_csv("files/score.csv")

""" final metrics"""

print(f"Accuracy: {acc_per_fold}")
print(f"F1: {f1_value_per_fold}")
print(f"Jaccard: {jac_value_per_fold}")
print(f"Recall: {recall_value_per_fold}")
print(f"Precision: {precision_value_per_fold}")
        
    ##########################

Train: 399 - 399
Valid: 132 - 132
Test: 132 - 132
Training for fold 1 ...
------------------------------------------------------------------------
Training for fold 1 ...
Train: 299 - 299
Valid: 132 - 132
Test: 100 - 100


2023-02-13 00:30:24.863601: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


(8, 1024, 1024, 3) (8, 1024, 1024, 1)
Epoch 1/260


2023-02-13 00:30:30.684355: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005
2023-02-13 00:30:40.654647: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.39GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


38/38 [==============================] - 55s 1s/step - loss: 0.8858 - accuracy: 0.3725 - dice_coef: 0.2716 - iou: 0.1587 - recall: 0.8360 - precision: 0.2902 - val_loss: 0.7074 - val_accuracy: 0.3519 - val_dice_coef: 0.2235 - val_iou: 0.1271 - val_recall: 0.6351 - val_precision: 0.2610

Epoch 00001: val_loss improved from inf to 0.70743, saving model to files/model.h5
Epoch 2/260
38/38 [==============================] - 33s 877ms/step - loss: 0.6632 - accuracy: 0.5923 - dice_coef: 0.3042 - iou: 0.1809 - recall: 0.7528 - precision: 0.4133 - val_loss: 0.6968 - val_accuracy: 0.3986 - val_dice_coef: 0.2242 - val_iou: 0.1275 - val_recall: 0.5095 - val_precision: 0.2450

Epoch 00002: val_loss improved from 0.70743 to 0.69683, saving model to files/model.h5
Epoch 3/260
38/38 [==============================] - 33s 878ms/step - loss: 0.5501 - accuracy: 0.7445 - dice_coef: 0.3351 - iou: 0.2029 - recall: 0.6637 - precision: 0.6286 - val_loss: 0.6764 - val_accuracy: 0.5378 - val_dice_coef: 0.2251 

...

eval metrics
Accuracy: 0.97258
F1: 0.87760
Jaccard: 0.80679
Recall: 0.88634
Precision: 0.89486
Training for fold 3 ...
------------------------------------------------------------------------
Training for fold 3 ...
Train: 299 - 299
Valid: 132 - 132
Test: 100 - 100
(8, 1024, 1024, 3) (8, 1024, 1024, 1)
Epoch 1/260
38/38 [==============================] - 37s 897ms/step - loss: 0.6563 - accuracy: 0.6053 - dice_coef: 0.2344 - iou: 0.1342 - recall_2: 0.5639 - precision_2: 0.3717 - val_loss: 0.7055 - val_accuracy: 0.1452 - val_dice_coef: 0.2283 - val_iou: 0.1302 - val_recall_2: 0.9682 - val_precision_2: 0.2619

Epoch 00001: val_loss improved from inf to 0.70554, saving model to files/model.h5
Epoch 2/260
38/38 [==============================] - 33s 877ms/step - loss: 0.5408 - accuracy: 0.7811 - dice_coef: 0.2796 - iou: 0.1643 - recall_2: 0.5774 - precision_2: 0.7370 - val_loss: 0.7060 - val_accuracy: 0.1602 - val_dice_coef: 0.2296 - val_iou: 0.1310 - val_recall_2: 0.9631 - val_precision_2:

100%|██████████| 100/100 [02:03<00:00,  1.23s/it]


eval metrics
Accuracy: 0.97694
F1: 0.90045
Jaccard: 0.83137
Recall: 0.90362
Precision: 0.91179
Training for fold 4 ...
------------------------------------------------------------------------
Training for fold 4 ...
Train: 300 - 300
Valid: 132 - 132
Test: 99 - 99
(8, 1024, 1024, 3) (8, 1024, 1024, 1)
Epoch 1/260
38/38 [==============================] - 42s 1s/step - loss: 0.4551 - accuracy: 0.7556 - dice_coef: 0.2723 - iou: 0.1596 - recall_3: 0.3124 - precision_3: 0.6205 - val_loss: 0.6799 - val_accuracy: 0.6908 - val_dice_coef: 0.2237 - val_iou: 0.1272 - val_recall_3: 0.2151 - val_precision_3: 0.3973

Epoch 00001: val_loss improved from inf to 0.67986, saving model to files/model.h5
Epoch 2/260
38/38 [==============================] - 34s 880ms/step - loss: 0.3839 - accuracy: 0.7717 - dice_coef: 0.3322 - iou: 0.2021 - recall_3: 0.4298 - precision_3: 0.7124 - val_loss: 0.6564 - val_accuracy: 0.7162 - val_dice_coef: 0.2252 - val_iou: 0.1281 - val_recall_3: 0.2152 - val_precision_3: 0.48

100%|██████████| 99/99 [01:59<00:00,  1.20s/it]

eval metrics
Accuracy: 0.97105
F1: 0.87503
Jaccard: 0.80577
Recall: 0.88877
Precision: 0.89234
Accuracy: [0.9713298892974853, 0.9725794219970703, 0.9769402408599853, 0.9710460142655806]
F1: [0.8702217714211433, 0.877600992584566, 0.9004501619910188, 0.8750327321014304]
Jaccard: [0.8031425948447217, 0.8067905796248089, 0.8313717067134622, 0.80577066681594]
Recall: [0.8829293480523234, 0.8863370738135462, 0.9036156221628525, 0.8887658263347461]
Precision: [0.9068583211038722, 0.8948569625764108, 0.9117851433252929, 0.8923447822444424]


In [7]:
print(f"Accuracy: {np.ceil(np.mean(acc_per_fold)*10000)/100}%")
print(f"F1: {np.ceil(np.mean(f1_value_per_fold)*10000)/100}%")
print(f"Jaccard: {np.ceil(np.mean(jac_value_per_fold)*10000)/100}%")
print(f"Recall: {np.ceil(np.mean(recall_value_per_fold)*10000)/100}%")
print(f"Precision: {np.ceil(np.mean(precision_value_per_fold)*10000)/100}%")

Accuracy: 97.3%
F1: 88.09%
Jaccard: 81.18%
Recall: 89.05%
Precision: 90.15%


In [8]:
    """
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'y', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('loss.png')
    plt.show()
    #plt.imshow(mpimg.imread('./loss.png'))

    accu = history.history['accuracy']
    val_accu = history.history['val_accuracy']
    epochs = range(1, len(accu) + 1)
    plt.plot(epochs, accu, 'r', label='accuracy') 
    plt.plot(epochs, val_accu, 'b', label='val_accuracy') 
    plt.title('accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('accuracy')
    plt.legend()
    plt.savefig('accuracy.png')
    plt.show()
    #plt.imshow(mpimg.imread('./accuracy.png'))
    """

"\nloss = history.history['loss']\nval_loss = history.history['val_loss']\nepochs = range(1, len(loss) + 1)\nplt.plot(epochs, loss, 'y', label='Training loss')\nplt.plot(epochs, val_loss, 'r', label='Validation loss')\nplt.title('Training and validation loss')\nplt.xlabel('Epochs')\nplt.ylabel('Loss')\nplt.legend()\nplt.savefig('loss.png')\nplt.show()\n#plt.imshow(mpimg.imread('./loss.png'))\n\naccu = history.history['accuracy']\nval_accu = history.history['val_accuracy']\nepochs = range(1, len(accu) + 1)\nplt.plot(epochs, accu, 'r', label='accuracy') \nplt.plot(epochs, val_accu, 'b', label='val_accuracy') \nplt.title('accuracy')\nplt.xlabel('Epochs')\nplt.ylabel('accuracy')\nplt.legend()\nplt.savefig('accuracy.png')\nplt.show()\n#plt.imshow(mpimg.imread('./accuracy.png'))\n"

In [9]:
"""
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
#import dice_coef, iou
#import load_data

H = 1024
W = 1024


def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x / 255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)  ## (1, 256, 256, 3)
    return ori_x, x


def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x / 255.0
    x = x > 0.5
    x = x.astype(np.int32)
    return ori_x, x


def save_result(ori_x, ori_y, y_pred, save_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)  ## (256, 256, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)  ## (256, 256, 3)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255.0

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_path, cat_images)


if __name__ == "__main__":
    create_dir("results")

    """ Load Model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
        model = tf.keras.models.load_model("files/model.h5")
        #model.summary()

    """ Dataset """
    dataset_path = "./Comp2018DatasetBiCubic"
    #dataset_path = "/imagesToAugment/images"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    """ Prediction and metrics values """
    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        name = x.split("/")[-1]

        """ Reading the image and mask """
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)

        """ Prediction """
        y_pred = model.predict(x)[0] > 0.5
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred.astype(np.int32)

        save_path = f"results/{name}"
        save_result(ori_x, ori_y, y_pred, save_path)

        """ Flattening the numpy arrays. """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculating metrics values """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
        SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

    """ Metrics values """
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    """ Saving all the results """
    df = pd.DataFrame(SCORE, columns=["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("files/score.csv")
    
  
    """


SyntaxError: invalid syntax (1488776625.py, line 59)

##################################""

In [ ]:
  '''
    """ K-Fold testing 10%"""
from sklearn.model_selection import KFold 
kf = KFold(n_splits=2) 
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
#model = tf.keras.models.load_model("files/model.h5") 
model = tf.keras.models.load_model("files/model.h5", custom_objects={'dice_coef': dice_coef, 'iou':iou}) 
#Generate the k-fold indices 
for train_index,test_index in kf.split(X):
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index], y[test_index]
    
    # Create and fit the model
    model.fit(X_train, y_train)
    # Evaluate the model
    accuracy = model.score(X_test, y_test)
    print('Fold accuracy: ', accuracy)'''

In [ ]:
 
'''
from sklearn.model_selection import KFold
import numpy as np

# define 4-fold cross validation test harness
kfold = KFold(n_splits=2) 
# enumerate splits
X= "/kaggle/working/files/data.csv"
df = pd.read_csv('/kaggle/input/passenger-list-for-the-estonia-ferry-disaster/estonia-passenger-list.csv')
for train_index, test_index in kfold.split(X):
    # create model
    model = "/kaggle/working/files/model.h5"
    # select rows for train and test
    print(train_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # fit model on train data
    model.fit(X_train, y_train, epochs=6, batch_size=32, verbose=0)
    # evaluate model on test data
    scores = model.evaluate(X_test, y_test, verbose=0)
    print(scores)'''

##################################""

In [ ]:
'''
import cv2
import glob
import matplotlib.pyplot as plt
    
path="./results/*.jpg"

images=[cv2.imread(image) for image in glob.glob(path)]
fig=plt.figure()
for i in range(len(images)):
    #plt.subplot(5,5,i+1)
    plt.imshow(images[i])
    plt.show()'''

In [ ]:
'''!zip -r ./file.zip ./results '''

In [ ]:
'''from IPython.display import FileLink
FileLink(r'file.zip')'''